In [1]:
import neo4j_connector as nc
import re
import docx
import os
from nltk.corpus import stopwords
from nltk.tag import StanfordPOSTagger
from nltk.tokenize import word_tokenize
import spacy
import pickle

In [2]:
# %run ./carga_diputados.ipynb

In [3]:
def extract_dialogs(doc):
    inicio = False
    dialog = ''
    dialogs = []
    for par in doc.paragraphs:   
        if par.text == '###':
            inicio = True
        else:
            inicio = False

        if inicio:
            if dialog == '':
                pass
            else:
                dialogs.append(dialog)
            dialog = ''
        else:
            dialog = dialog + par.text + ' '
    return dialogs

In [4]:
def separate_dialogs(dialogs):
    output = []
    for dialog in dialogs:
        pos_fin = dialog.find(':')
        output.append([dialog[:pos_fin].lower(), dialog[pos_fin + 1 :].lower()])
    return output

In [5]:
def clean_parenthesis(text):
    regex = r" \(([^\)]+)\)"
    matches = re.finditer(regex, text)
    output = text
    for matchNum, match in enumerate(matches, start=1):
        t_parentesis = match.group()
        output = output.replace(t_parentesis, '')
    return output

In [6]:
def clean_mr_mrs(text):
    return text.replace('el señor ', '').replace('la señora ', '').replace('señor ', '').replace('señora ', '')

In [7]:
def clean_text(text):
    return text.replace('?', '').replace('¿', '').replace('«', '').replace('»', '') \
.replace('.', ' ').replace(',', ' ').replace('—', ' ').replace('-', ' ').replace('%', ' ') \
.replace('š', ' ').replace(';', ' ').replace('!', ' ').replace('¡', ' ') \
.replace('–', ' ').replace(' : ', ': ').replace(':', ' ').replace('  ', ' ')

In [8]:
def clean_names(name):
    return name.replace('presidente del gobierno en funciones', 'sánchez pérez-castejón') \
        .replace('ministra de justicia en funciones', 'delgado garcía') \
        .replace('ministra de hacienda en funciones', 'montero cuadrado') \
        .replace('ministro del interior en funciones', 'grande-marlaska gómez') \
        .replace('rodrí guez hernández', 'rodríguez hernández') \
        .replace('ministro de agricultura pesca y alimentación en funciones', 'planas puchades') \
        .replace('ministro de fomento en funciones', 'ábalos meco') \
        .replace('ministra de economía y empresa en funciones', 'calviño santamaría') \
        .replace(' candidato a la presidencia del gobierno', '') \
        .replace('ministro de inclusión  seguridad social y migraciones', 'escrivá belmonte') \
        .replace('ministra de política territorial y función pública', 'darias san sebastián') \
        .replace('ministra de hacienda', 'montero cuadrado') \
        .replace('representante de la asamblea regional de murcia', 'conesa alcaraz') \
        .replace('de olano vela', 'olano vela') \
        .replace('ministro del interior', 'grande-marlaska gómez') \
        .replace('ministro de interior', 'grande-marlaska gómez') \
        .replace('ministro de agricultura pesca y alimentación', 'planas puchades') \
        .replace('ministro de transportes movilidad y agenda urbana', 'ábalos meco') \
        .replace('presidente del gobierno', 'sánchez pérez-castejón') \
        .replace('ministro de sanidad', 'illa roca') \
        .replace('concicao', 'garriga vaz de concicao') \
        .replace('oblanca', 'martínez oblanca') \
        .replace('ministra de igualdad', 'montero gil') \
        .replace('herrer', 'rodríguez herrer') \
        .replace('ministra de industria comercio y turismo', 'maroto illera') \
        .replace('ministro de inclusión seguridad social y migraciones', 'escrivá belmonte') \
        .replace('moro', 'oramas gonzález-moro') \
        .replace('ministra de asuntos exteriores unión europea y cooperación', 'gonzález laya') \
        .replace('ministro de justicia', 'campo moreno') \
        .replace('vicepresidente segundo del gobierno y ministro de derechos sociales y agenda 2030', 'iglesias turrión') \
        .replace('martínez martínez oblanca', 'martínez oblanca')

In [9]:
def remove_stopwords(dialog):
    list_specific_stop_words = ['gracias', 'señor', 'señora', 'señorías', 'presidenta', 'ministro', 'ustedes',
                                'usted', 'si', ]
    stop_words = set(stopwords.words('spanish'))

    word_tokens = word_tokenize(dialog)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [w for w in filtered_sentence if not w in list_specific_stop_words]
        
    return filtered_sentence

In [10]:
def clean_dialogs(dialogs):
    output = []
    for dialog in dialogs:
        speaker = clean_names(clean_mr_mrs(clean_parenthesis(dialog[0])))
        text = remove_stopwords(clean_text(clean_parenthesis(dialog[1])))
        output.append([speaker, text])
    return output

In [11]:
# Almacenamos en una lista, la relacion y la lista de diálogos
congreso = []
for file in os.listdir('files/'):
    if file[-4:] == 'docx':
        doc = docx.Document('files/' + file) 
        dialogs = extract_dialogs(doc)
        sep_dialogs = separate_dialogs(dialogs)
        cl_dialogs = clean_dialogs(sep_dialogs)
        relation = file[:-5]
        congreso.append([relation, cl_dialogs])

#### En este momento tenemos una estructura de este tipo:
['ID_DEBATE', [[DIPUTADO, [PALABRAS]]]]'

In [ ]:
def generate_tagger_model()